# Leitura inicial - dados IBGE
## objetivo: 
Padronizar, limpar e organizar os dados de bairros de Belém-PA a partir de múltiplos arquivos CSV

### Resumo Descritivo do Código

O código realiza a leitura de múltiplos arquivos CSV contendo dados de bairros de Belém-PA, tanto do conjunto `tabela3033` quanto do `tabela3170`. Após a leitura:

#### 1. Limpeza dos nomes de bairros
- Remove o sufixo `- Belém (PA)` das colunas `Bairro` em todos os DataFrames, garantindo uniformidade.

#### 2. Verificação de consistência
- Compara as listas de bairros entre diferentes DataFrames para identificar discrepâncias ou bairros ausentes.

#### 3. Salvamento seguro dos arquivos
- Cria uma função `salvar_csv` que grava os DataFrames em arquivos CSV no diretório de processamento.
- Evita sobrescrever arquivos já existentes e informa o status do salvamento.

**Resumo geral:**  
O código prepara e padroniza os dados de bairros para posterior análise, garantindo que os nomes estejam consistentes e os arquivos processados sejam salvos de forma organizada.



In [1]:
import json
import pandas as pd
import os

## 1. Open files

In [2]:
path="/home/akel/PycharmProjects/Data_ambiental/data/raw/"
filename_a='tabela3033_a.csv'
filename_b='tabela3033_b.csv'
filename_c='tabela3033_c.csv'

dfa = pd.read_csv(path+filename_a,sep=';',skiprows=4)
dfb = pd.read_csv(path+filename_b,sep=';',skiprows=4)
dfc = pd.read_csv(path+filename_c,sep=';',skiprows=4)

filename_A='tabela3170_a.csv'
filename_B='tabela3170_b.csv'
filename_C='tabela3170_c.csv'
filename_D='tabela3170_d.csv'

dfA = pd.read_csv(path+filename_A,sep=';',skiprows=5)
dfB = pd.read_csv(path+filename_B,sep=';',skiprows=5)
dfC = pd.read_csv(path+filename_C,sep=';',skiprows=5)
dfD = pd.read_csv(path+filename_D,sep=';',skiprows=5)
#dfe = pd.read_csv(path+filename_e,sep=';',skiprows=4)


## 2. Removendo sufixo

In [3]:
#removendo o sufixo da variavel bairro  - Belém (PA)
dfa['Bairro'] = dfa['Bairro'].str.replace(' - Belém (PA)', '', regex=False).str.strip()
dfb['Bairro'] = dfb['Bairro'].str.replace(' - Belém (PA)', '', regex=False).str.strip()
dfc['Bairro'] = dfc['Bairro'].str.replace(' - Belém (PA)', '', regex=False).str.strip()

dfA['Bairro'] = dfA['Bairro'].str.replace('- Belém (PA)', '', regex=False).str.strip()
dfB['Bairro'] = dfB['Bairro'].str.replace('- Belém (PA)', '', regex=False).str.strip()
dfC['Bairro'] = dfC['Bairro'].str.replace('- Belém (PA)', '', regex=False).str.strip()
dfD['Bairro'] = dfD['Bairro'].str.replace('- Belém (PA)', '', regex=False).str.strip()


## 3. verificando se os bairros são os mesmos nas duas tabelas.

In [4]:
#Verificar se são os mesmo bairros
a=dfa['Bairro'].unique().tolist()
A=dfA['Bairro'].unique().tolist()
print(list(set(A) - set(a)))

[]


In [14]:
# juntando as 3 tabelas 3033a,b,c
#a-Número de domicílios particulares ocupados
#b-Pessoas residentes em domicílios particulares
#c-Média de moradores em domicílios particulares ocupados

dfa_r=dfa.rename(columns={
    "Total": "Habitações"
})
dfb_r=dfb.rename(columns={
    "Total": "Moradores"
})
dfc_r=dfc.rename(columns={
    "Total": "Média"
})

dfa_r = dfa_r[["Bairro", "Habitações"]]
dfb_r = dfb_r[["Bairro", "Moradores"]]
dfc_r = dfc_r[["Bairro", "Média"]]

df_final=dfa_r.copy()
df_final['Moradores']=dfb_r['Moradores']
df_final['Média']=dfc_r['Média']
df_final.head(3)
df_final["Média"] = df_final["Média"].str.replace(",", ".", regex=False).astype(float)

df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Bairro      71 non-null     object 
 1   Habitações  71 non-null     int64  
 2   Moradores   71 non-null     int64  
 3   Média       71 non-null     float64
dtypes: float64(1), int64(2), object(1)
memory usage: 2.3+ KB


In [12]:
# juntando as 4 tabelas 3170 A,B,C D
dfA_r=dfA.rename(columns={
    "Total": "N",
    "Total.1":'N Homens',
    "Total.2": 'N Mulheres'})

dfB_r=dfB.rename(columns={
    "Total": "P",
    "Total.1":'P Homens',
    "Total.2": 'P Mulheres'})

dfC_r=dfC.rename(columns={
    "Total": "avg",
    "Total.1":'avg Homens',
    "Total.2": 'avg Mulheres'})

dfD_r=dfD.rename(columns={
    "Total": "mdn",
    "Total.1":'mdn Homens',
    "Total.2": 'mdn Mulheres'})

DF_FINAL=dfA_r.copy()
DF_FINAL["P"] = dfB_r["P"]
DF_FINAL["P Homens"] = dfB_r["P Homens"]
DF_FINAL["P Mulheres"] = dfB_r["P Mulheres"]

DF_FINAL["avg"] = dfC_r["avg"]
DF_FINAL["avg Homens"] = dfC_r["avg Homens"]
DF_FINAL["avg Mulheres"] = dfC_r["avg Mulheres"]

DF_FINAL["mdn"] = dfD_r["mdn"]
DF_FINAL["mdn Homens"] = dfD_r["mdn Homens"]
DF_FINAL["mdn Mulheres"] = dfD_r["mdn Mulheres"]

# Lista de colunas não numéricas
nao_numericas = ["Bairro", "Grupo de idade"]
temp=DF_FINAL.copy()

# Para todas as outras colunas:
for col in temp.columns:
    if col not in nao_numericas:
        temp[col] = temp[col].str.replace(",", ".", regex=False)
        temp[col] = pd.to_numeric(temp[col], errors="coerce")

DF_FINAL=temp.copy()
DF_FINAL.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1065 entries, 0 to 1064
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Bairro          1065 non-null   object 
 1   Grupo de idade  1065 non-null   object 
 2   N               1058 non-null   float64
 3   N Homens        1055 non-null   float64
 4   N Mulheres      1049 non-null   float64
 5   P               1058 non-null   float64
 6   P Homens        1055 non-null   float64
 7   P Mulheres      1049 non-null   float64
 8   avg             1058 non-null   float64
 9   avg Homens      1055 non-null   float64
 10  avg Mulheres    1049 non-null   float64
 11  mdn             1058 non-null   float64
 12  mdn Homens      1055 non-null   float64
 13  mdn Mulheres    1049 non-null   float64
dtypes: float64(12), object(2)
memory usage: 116.6+ KB


## 4. Salvando os novos dados

In [16]:
# salvar os novos dados
def salvar_csv(df, caminho):
    if os.path.exists(caminho):
        print(f"⚠️ Arquivo já existe e NÃO será sobrescrito: {caminho}")
    else:
        df.to_csv(caminho, index=False)
        print(f"✅ Arquivo salvo: {caminho}")

path = '/home/akel/PycharmProjects/Data_ambiental/data/process/'
salvar_csv(df_final, path + 'tabela3033_final.csv')
salvar_csv(DF_FINAL, path + 'tabela3170_final.csv')

✅ Arquivo salvo: /home/akel/PycharmProjects/Data_ambiental/data/process/tabela3033_final.csv
✅ Arquivo salvo: /home/akel/PycharmProjects/Data_ambiental/data/process/tabela3170_final.csv
